In [13]:
import os, configparser, random
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
#Actor Picker
def actor_picker():
    actors = ["Chris Evans", "Scarlett Johansson", "Hritik Roshan", "Emma Watson", "Tom Holland", "Akshay Kumar", "Srinidhi Shetty", "Ram Charan"]
    return random.choice(actors)


In [5]:
#Location Picker
def location_picker():
    locations = ["New York", "Mumbai", "Pune", "Solapur", "Los Angeles", "Washington DC", "Delhi", "SambhajiNagar"]
    return random.choice(locations)


In [6]:
#Theme Picker
def theme_picker():
    themes = ["Adventure", "Drama", "Thiller", "Horror", "Mystery", "Romance", "Science Fiction"]
    return random.choice(themes)


In [7]:
#Configure and Use Cohere Model
config = configparser.ConfigParser()
config.read('../Config.ini')
cohere = config['cohere']
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')


def cohere_story_generator(actor, location, theme):
    model = ChatCohere(model="command-r-plus")
    messages = [
        SystemMessage(content="You are a storyteller who crafts immersive and vivid tales."),
        HumanMessage(content=f"Create a brief story with {actor} in {location} based on {theme} theme.")
        ]
    response = model.invoke(messages)
    return response.content.strip()

In [8]:
#Configure and Use Groq Model
config = configparser.ConfigParser()
config.read('../Config.ini')
groq = config['groq']
os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')


def groq_story_generator(actor, location, theme):
    model = ChatGroq(model="llama3-8b-8192")
    messages = [
        SystemMessage(content="You are a storyteller who crafts immersive and vivid tales."),
        HumanMessage(content=f"Create a brief story with {actor} in {location} based on {theme} theme.")
        ]
    response = model.invoke(messages)

    return response.content.strip()

In [9]:
actor = actor_picker()
location = location_picker()
theme = theme_picker()

print(f"Selected Elements ==>")
print(f"Actor: {actor}\nLocation: {location}\nTheme: {theme}")

print("="*30)
print("Story from Cohere Model ==>")
print("="*30)
cohere_story = cohere_story_generator(actor, location, theme)
print(cohere_story)

print("="*30)
print("Story from Groq Model ==>")
print("="*30)
groq_story = groq_story_generator(actor, location, theme)
print(groq_story)

Selected Elements ==>
Actor: Tom Holland
Location: Solapur
Theme: Romance
Story from Cohere Model ==>
Once upon a time, in the vibrant city of Solapur, India, a young and dashing Tom Holland found himself enchanted by a local beauty named Priya. Tom, known for his adventurous spirit, was in Solapur to film his latest action-packed movie. Little did he know that he was about to embark on a different kind of adventure—one that would capture his heart forever.

As Tom explored the bustling streets of Solapur, he often found himself drawn to the local markets, where vibrant colors and aromatic spices filled the air. It was in one of these markets that he first laid eyes on Priya. She was a local artist, with dark, mysterious eyes and a smile that could light up the entire city. Tom felt an instant connection to her as if something magical was pulling them together.

Day after day, Tom found excuses to return to the market, hoping to catch a glimpse of Priya. He loved how her eyes lit up wh

In [10]:
model = ChatGroq(model="llama3-8b-8192")
parser = StrOutputParser()

chain = model | parser
response = chain.invoke([HumanMessage(content=input("Enter Message: "))])

print(response)

Nice to meet you, Pranav! I'm GPT, your friendly AI assistant. How can I help you today? Do you have a specific topic you'd like to discuss or a question you'd like to ask? I'm all ears (or rather, all text)!


In [11]:
chain = RunnableSequence(model, parser)
chain.invoke([HumanMessage(content=input("Enter message: "))])

"Hello Pranav! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [14]:
# import the chat history classes
from langchain_core.chat_history import (BaseChatMessageHistory, InMemoryChatMessageHistory)
import asyncio

store = InMemoryChatMessageHistory()
async def getMessage():
    await asyncio.sleep(2)
    print("Message retrieved from DB")
    return await store.aget_messages()


store.add_message(HumanMessage(input("Enter message: ")))
messages = await(getMessage())
response = model.invoke(messages) 
print(response.content)

store.add_message(SystemMessage(response.content))
store.add_message(HumanMessage('Lets see if you know my name?'))

messages = await(getMessage())
print(messages) 

response = model.invoke(messages)
print(response.content) 

Message retrieved from DB
**The Unlikely Rise of Yash: From Struggle to Stardom**

In a small town in Karnataka, India, a young boy named Naveen Kumar Gowda was born to a middle-class family. Little did anyone know that this boy would grow up to become one of the most iconic actors in Indian cinema, known as Yash.

Growing up, Yash was a shy and introverted child who struggled to find his place in the world. His parents, who were both teachers, encouraged him to pursue his passion for acting, but it wasn't easy. Yash's early attempts to make it big in the film industry were met with rejection and disappointment.

After completing his education, Yash moved to Bangalore to pursue a career in acting. He landed small roles in TV commercials and short films, but it was a struggle to make ends meet. He worked multiple jobs to support himself, including as a salesperson and a waiter, all while pursuing his acting dreams.

Despite his best efforts, Yash faced rejection after rejection. He was 

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store: 
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
config = {'configurable': {"session_id": "abc2"}}
withHistory = RunnableWithMessageHistory(model, get_session_history)

response = withHistory.invoke([HumanMessage(content=input("Enter message: "))], config=config)

print(response.content)

response = withHistory.invoke(
    [HumanMessage(content=input("Enter msg to check memory: "))], config=config)

print(response.content)

This conversation has just started, so you haven't sent any messages yet! What would you like to talk about? I can summarize our conversation at the end if you'd like.
In that case, your last message is indeed this one: "this is my last message".


In [16]:
withHistory = RunnableWithMessageHistory(model, get_session_history)
response = withHistory.invoke([HumanMessage(content=input("Enter message: "))], config=config)

print(response.content) 

response = withHistory.invoke(
    [HumanMessage(content=input("Enter msg to check memory: "))], config=config
)

print(response.content)

I see what you did there! You're asking me to repeat what I said earlier, which is that your last message is indeed this one: "this is my last message".
It looks like you're trying to ask me something without typing any text!


In [ ]:
while True:
    response = withHistory.invoke(
        [HumanMessage(content=input("Enter msg to check memory: "))], config=config
    )

    print(response.content)